## duplicate this notebook and write to that one

In [22]:
# (CPU, DataSize, Y=pick from columns and iterate)
import pandas as pd

In [23]:
df = pd.read_csv("testing.csv", sep=",") #data from 22July2016 
# copy the 'dup' folder to ~/ProfilingData/.
# remove *Output* files 
# run "python extract.py dup/" to generate testing.csv

In [24]:
len(df)

32

In [25]:
df.head(n=1)

,ppn,mem,seqlen,trial,AvgRSS,AvgSharedTxt,AvgStack,AvgTotal,AvgUnsharedData,CommandTimed,...,PageSize,PercentCPU,SignalsDeliv,SocketMsgRecv,SocketMsgSent,Swaps,SystemTime,UserTime,VolCtxtSwitch,WallTime
0,11,64G,10M,1,0,0,0,0,0,./NGS-ann-program/BIN/cd-hit-dup -i ./data/no-...,...,4096,99%,0,0,0,0,3.5,42.32,1072,0:45.97


In [26]:
#find which columns are completely empty
a=len(df[df['AvgRSS'] > 0])
b=len(df[df['AvgSharedTxt'] > 0])
c=len(df[df['AvgStack'] > 0])
d=len(df[df['AvgTotal'] > 0])
e=len(df[df['AvgUnsharedData'] > 0])

print(a,b,c,d,e)

0 0 0 0 0


In [27]:
header = list(df.columns)

In [28]:
i = 1
a = 0
emptyColumns = []
for h in header:
    try:
        a=len(df[df[h] > 0])
        print(a,  h)
        if(a==0): #nothing is greater than zero for this column
            emptyColumns.append(h)
    except:
        print('......................'+ h +' not a scalar')
    i=i+1

32 ppn
......................mem not a scalar
......................seqlen not a scalar
32 trial
0 AvgRSS
0 AvgSharedTxt
0 AvgStack
0 AvgTotal
0 AvgUnsharedData
......................CommandTimed not a scalar
0 ExitStatus
32 FileSysIn
32 FileSysOut
32 InvolCtxtSwitch
32 MajorFaults
32 MaxRSS
32 MinorFaults
32 PageSize
......................PercentCPU not a scalar
0 SignalsDeliv
0 SocketMsgRecv
0 SocketMsgSent
0 Swaps
32 SystemTime
32 UserTime
32 VolCtxtSwitch
......................WallTime not a scalar


In [29]:
emptyColumns.append('trial')
emptyColumns.append('PageSize')
emptyColumns.append('CommandTimed')
emptyColumns.append('WallTime')

In [30]:
for c in emptyColumns:
    del df[c]

In [31]:
df.head(5)

,ppn,mem,seqlen,FileSysIn,FileSysOut,InvolCtxtSwitch,MajorFaults,MaxRSS,MinorFaults,PercentCPU,SystemTime,UserTime,VolCtxtSwitch
0,11,64G,10M,3234924,2179426,212,3,6544812,1388550,99%,3.50,42.32,1072
1,11,64G,15M,4915882,3320366,608,3,9621592,2150793,99%,5.43,62.54,1634
2,11,64G,20M,6599917,4467723,366,3,13416152,2892383,99%,7.44,91.03,2228
3,11,64G,25M,8250892,5591773,4795,3,16387764,3612806,99%,9.36,111.08,2746
4,11,64G,30M,9788169,6637009,164,3,19447720,4284645,99%,11.16,130.50,3264


In [32]:
ft = df.copy(deep=True)

In [33]:
delthis = list(set(ft.columns.values) - set(['ppn','seqlen','VolCtxtSwitch']))
delthis

['MinorFaults',
 'MajorFaults',
 'InvolCtxtSwitch',
 'FileSysIn',
 'UserTime',
 'SystemTime',
 'FileSysOut',
 'mem',
 'MaxRSS',
 'PercentCPU']

In [34]:
for c in delthis:
    del ft[c]

In [35]:
#DROP the lines which are NULL
#df = df[df.CommandTimed.notnull()]
len(df)

32

In [36]:
whatisthesize = {'5M': .54, '10M': 1.2,'15M': 1.7,'20M': 2.3,'25M': 2.9,'30M': 3.4,'35M': 4.0,'40M': 4.5}

In [37]:
ft['sizeGB'] = ft['seqlen'].map(whatisthesize)

In [38]:
del ft['seqlen']
ft.head(2)

,ppn,VolCtxtSwitch,sizeGB
0,11,1072,1.2
1,11,1634,1.7


In [39]:
ft['Y_VolCtxt']=ft['VolCtxtSwitch']
del ft['VolCtxtSwitch']
ft.head()

,ppn,sizeGB,Y_VolCtxt
0,11,1.2,1072
1,11,1.7,1634
2,11,2.3,2228
3,11,2.9,2746
4,11,3.4,3264


In [40]:
from sklearn.ensemble.forest import RandomForestRegressor
import numpy as  np

In [72]:
model = RandomForestRegressor(n_estimators=100, n_jobs=-1)

In [87]:
np.random.seed()
ft['is_train']=np.random.uniform(0,1,len(ft)) <= .70
train, test = ft[ft['is_train']==True], ft[ft['is_train']==False]
del ft['is_train']
len(train)*1.0 / (len(test)+len(train))

0.71875

In [88]:
ft.columns[0:2]

Index(['ppn', 'sizeGB'], dtype='object')

In [89]:
ft.columns[2]

'Y_VolCtxt'

In [90]:
features = ft.columns[0:2]
target = ft.columns[2]
model.fit(train[features], train[target].values)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [91]:
Y_prediction = model.predict(test[features])
Y_test = test[target].as_matrix()

In [92]:
Y_test #Print Y values of test dataset

array([3264, 1066, 2199, 3246, 4298, 1645, 2215, 2157,  516])

In [93]:
Y_prediction #Print Y predicted values of test dataset

array([ 3137.17,  1051.73,  1946.52,  3179.21,  4288.59,  1552.36,
        1944.31,  1980.58,   597.54])

In [94]:
model.score(test[features], test[target]) #which score is this ?

0.98141412887192403

In [95]:
from sklearn import cross_validation
scores = cross_validation.cross_val_score(model, ft[features], ft[target], cv=10,scoring='mean_squared_error')

In [96]:
scores

array([-1776.59325   , -2953.8079    ,  -508.3462    , -3200.7159    ,
        -544.1403    , -4501.55913333, -1982.84086667, -1288.93146667,
        -648.0158    , -1293.33686667])

In [97]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: -1869.83 (+/- 2503.39)


In [98]:
ft[target].mean(), ft[target].std() * 2

(2441.5625, 2526.97656588187)

In [99]:
scores = cross_validation.cross_val_score(model, ft[features], ft[target], cv=10,scoring='r2')
print("Accuracy: %0.8f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.99384865 (+/- 0.01)
